In [33]:
from patcher.extractor import FateSeeker1Patcher, FateSeeker1PatchHelper, FateSeeker1MetaInfo
import UnityPy
import pathlib
import os

In [34]:
import csv

In [ ]:
FateSeeker1MetaInfo.dlc_opening

In [36]:
backuped_assets_path = pathlib.Path("extracted_assets")

In [37]:
fate_patcher = FateSeeker1Patcher(pathlib.Path(os.path.abspath(".")).joinpath("extracted_assets"))
fate_pach_helper = FateSeeker1PatchHelper(fate_patcher)

In [65]:
t = fate_patcher.get_text("assets/forassetbundles/textfiles/localization.csv")

In [ ]:
t[:100]

In [39]:
csv_reader = csv.reader(t.splitlines(), delimiter=',')

In [40]:
from io import StringIO
class CustomCsvDict:

    def __init__(self, csv_text: str) -> None:
        self.csv = self.heoluo_string_csv_parser(csv_text)

    @staticmethod
    def heoluo_string_csv_parser(text: str) -> list[dict[str, str]]:
        output = list()
        headers = None
        for line in text.split("\n"):
            if line.endswith("\r"):
                line = line[:-1]
            if line.startswith("#"):
                print("comment find")
                print(line)
                continue
            if not headers:
                headers = line.split("\t")
                continue
            if line == "":
                continue
            output.append(dict(zip(headers, line.split("\t"))))
        return output

    def to_csv_string(self) -> str:
        headers = self.csv[0].keys()
        output = StringIO()
        output.write("\t".join(headers))
        output.write("\r\n")
        for row in self.csv:
            output.write("\t".join(row.values()))
            output.write("\r\n")
        return output.getvalue()

    def __getitem__(self, key: str) -> dict[str, str]:
        return self.csv[key]


In [ ]:
for line in csv_reader:
    print(line)

In [41]:
cd = CustomCsvDict(t)

In [42]:
text = cd.to_csv_string()

In [ ]:
cd[:100]

In [ ]:
for t in cd[:5]:
    print(t)

In [15]:
fp.set_text("assets/forassetbundles/textfiles/localization.csv", "qweqweqweqwe")
fp.save_asset("./test")